In [ ]:
import datetime

from src.base.forecasting.models import Activation, LrMaxCriterion, LogLogAUCLoss, MSELoss
from src.base.forecasting.models.time_series import TimeSeriesModelAutoRegressiveMLP

from src.projects.fagradalsfjall.common.dataset import load_train_data_numpy
from src.projects.fagradalsfjall.common.model_eval import ModelEvalResult
from src.projects.fagradalsfjall.common.model_repo import ModelRepo
from src.projects.fagradalsfjall.common.project_settings import (
    CV_HORIZON_SAMPLES,
    TS_CV_SPLITTER,
    TS_PRIMARY_METRIC,
)
from src.projects.fagradalsfjall.blog_6_nn.param_grids import params_informed_search

# 1. Smart Search across all parameters (including loss, batch size, ...)

In [ ]:
# -------------------------------------------------------------------------
#  Load TRAINING set
# -------------------------------------------------------------------------

print("Loading training dataset...    ", end="")
x_train = load_train_data_numpy()
print("Done.")

In [ ]:
# -------------------------------------------------------------------------
#  MLP - Some test runs without CV
# -------------------------------------------------------------------------

# --- define model ----------------------------------------
model_mlp = TimeSeriesModelAutoRegressiveMLP(p=64, n=192, batch_size=512)
models = {"mlp": model_mlp}

# --- configure param optimization ------------------------
param_grid = params_informed_search

deadline = datetime.datetime(2022, 10, 7, 7, 0, 0)  # Fri morning 7am
max_seconds = (deadline - datetime.datetime.now()).total_seconds()
init_iters = 108
max_iter = 4 * 108
n_jobs = 12

# --- fit - cross-validation ------------------------------
model_mlp.cv.line_search(
    x=x_train,
    param_grid=param_grid,
    metric=TS_PRIMARY_METRIC,
    ts_cv_splitter=TS_CV_SPLITTER,
    hor=CV_HORIZON_SAMPLES,
    retrain=True,
    max_seconds=max_seconds,
    max_iter=max_iter,
    n_jobs=n_jobs,
    closest_first=True,
    init_iters=init_iters,
)

In [ ]:
# plot grid search results
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="activation").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="batch_size").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="lr_max").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="n_hidden_layers").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="layer_width").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="dropout").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="wd").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="p").create(w=18, h=12)
model_mlp.cv.results.plot_2d(x_param="n_epochs", y_param="n").create(w=18, h=12)

model_mlp.cv.results.plot_1d(param_names="activation").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="lr_max").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n_epochs").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="batch_size").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n_hidden_layers").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="layer_width").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="dropout").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="wd").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="p").create(w=18, h=12)
model_mlp.cv.results.plot_1d(param_names="n").create(w=18, h=12)

In [ ]:
# --- evaluate & save -------------------------------------
model_eval_results = ModelEvalResult.eval_many(models, compute_metrics=True)

ModelRepo.save_models(model_eval_results)